In [2]:
import sys
sys.path.append("C:/dev/akshare/")
sys.path.append("C:/dev/pyktrader3/")
import numpy as np
import pandas as pd
import requests
from io import BytesIO, StringIO
import datetime
from pycmqlib3.utility.misc import day_shift, CHN_Holidays
from akshare.futures import cons
from misc_scripts.aks_data_update import save_data
from pycmqlib3.utility.misc import CHN_Holidays, day_shift, is_workday, product_code, instID_adjust

In [5]:
# http://www.dce.com.cn/publicweb/quotesdata/exportDayQuotesChData.html&year=2021&month=3&day=7&exportFlag=excel"
url = "http://www.dce.com.cn/publicweb/quotesdata/exportDayQuotesChData.html"
headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Content-Length": "86",
        "Content-Type": "application/x-www-form-urlencoded",
        "Host": "www.dce.com.cn",
        "Origin": "http://www.dce.com.cn",
        "Pragma": "no-cache",
        "Referer": "http://www.dce.com.cn/publicweb/quotesdata/dayQuotesCh.html",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36",
    }
date='20210406'
params = {
        "dayQuotes.variety": "all",
        "dayQuotes.trade_type": "0",
        "year": date[:4],
        "month": str(int(date[4:6]) - 1),
        "day": date[6:],
        "exportFlag": "excel",
    }
session = requests.Session()
r = session.get(url, data=params)
xdf = pd.read_excel(BytesIO(r.content))
print(xdf)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [28]:
fut_table = 'fut_daily'
flavor = 'mysql'
exch = 'DCE'
end_date=datetime.date(2021,4,16)
date = end_date.strftime("%Y%m%d")

#data_df = pd.read_excel("C:\\Users\\harve\\Downloads\\%s_Daily.xls" % (date))
data_df = pd.read_csv("C:\\Users\\harve\\Downloads\\%s_Daily.txt" % (date), delimiter='\s+')

data_df = data_df[~data_df["商品名称"].str.contains("小计")]
data_df = data_df[~data_df["商品名称"].str.contains("总计")]
data_df["variety"] = data_df["商品名称"].map(lambda x: cons.DCE_MAP[x])
data_df["symbol"] = data_df["variety"] + data_df["交割月份"].astype(int).astype(str)
del data_df["商品名称"]
del data_df["交割月份"]
data_df.columns = ["open", "high", "low", "close",
                   "pre_settle", "settle", "_", "_",
                   "volume", "open_interest", "_", "turnover", "variety", "symbol"]
data_df["date"] = date
data_df = data_df[
    ["symbol", "date", "open", "high", "low", "close", "volume", "open_interest", "turnover", "settle",
     "pre_settle", "variety"]]
data_df = data_df.applymap(lambda x: x.replace(",", ""))
data_df = data_df.astype({"open": "float",
                          "high": "float",
                          "low": "float",
                          "close": "float",
                          "volume": "float",
                          "open_interest": "float",
                          "turnover": "float",
                          "settle": "float",
                          "pre_settle": "float",
                          })
print(data_df)    

    symbol      date    open    high     low   close    volume  open_interest  \
0    A2105  20210416  5880.0  5922.0  5830.0  5893.0  118106.0        51517.0   
1    A2107  20210416  5907.0  5942.0  5803.0  5930.0   12346.0         8007.0   
2    A2109  20210416  5890.0  5952.0  5877.0  5944.0   52964.0        48110.0   
3    A2111  20210416  5750.0  5855.0  5750.0  5811.0    5413.0        16807.0   
4    A2201  20210416  5730.0  5751.0  5693.0  5733.0     720.0         5040.0   
..     ...       ...     ...     ...     ...     ...       ...            ...   
233  Y2109  20210416  8102.0  8296.0  8074.0  8248.0  673205.0       443831.0   
234  Y2111  20210416  7794.0  8130.0  7794.0  8084.0   26225.0        37788.0   
235  Y2112  20210416  7902.0  8074.0  7834.0  8040.0    7466.0        33594.0   
236  Y2201  20210416  7840.0  8024.0  7840.0  7972.0    8898.0        18872.0   
237  Y2203  20210416  7784.0  7902.0  7764.0  7864.0    1341.0         1638.0   

       turnover  settle  pr

In [29]:
df = data_df
if (df is not None) and (len(df) > 0):
    df = df[df['close'].apply(lambda x: pd.api.types.is_number(x))]
    df = df[df['open'].apply(lambda x: pd.api.types.is_number(x))]
    df = df[df['volume'].apply(lambda x: pd.api.types.is_number(x))]
    df = df[df['volume'] > 0]
    df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(str(x), "%Y%m%d").date())
    if exch in ['DCE', 'SHFE', 'INE']:
        df['symbol'] = df['symbol'].apply(lambda x: x.lower())
        df['variety'] = df['variety'].apply(lambda x: x.lower())
    if exch == 'SHFE':
        df = df[~df['variety'].apply(lambda x: x in product_code['INE'])]
    df['exch'] = exch
    df.rename(columns = {'symbol': 'instID', 'open_interest': 'openInterest'}, inplace=True)
    df['instID'] = df['instID'].apply(lambda x: instID_adjust(x, exch, end_date))
    xdf = df[['instID', 'exch', 'date', 'open', 'high', 'low', 'close', 'settle', 'volume', 'openInterest']]
    save_data(fut_table, xdf, flavor = flavor)

C:\Anaconda3\lib\site-packages\sqlalchemy\sql\base.py:299: SAWarning: Can't validate argument 'replace_string'; can't locate any SQLAlchemy dialect named 'replace'
  util.warn(
